In [1]:
# Debug Abaqus

In [131]:
import numpy as np
import tensorflow as tf
from misc import preprocessing, normalization
import tensorflow.keras.backend as K

dataset_name = 'P12AC1_xy'
model_name = 'P12AC1_xy'
model_fname   = 'savednet/'+model_name+'.json'
weights_fname = 'savednet/'+model_name+'_weights.h5'
model = tf.keras.models.model_from_json(open(model_fname).read())
model.load_weights(weights_fname)
ndata, I1, I2, Ia, Is, Psi_gt, X, Y, sigma_gt, F, C, C_inv  = preprocessing(dataset_name)
meanPsi, meanI1, meanI2, meanIa, meanIs, stdPsi, stdI1, stdI2, stdIa, stdIs = normalization(True, model_name, Psi_gt, I1, I2, Ia, Is)

In [132]:
# dfgrd1 = np.array([
#     [1.05181164390228,         0.279726191613219,       -7.170170769637634E-002],
#     [-7.968914041957097E-002,  0.898491253723326,       -2.073957440069644E-003],
#     [1.755877436359610E-002,  -4.041710028534459E-003,   1.04254149792529      ]]
# ,dtype=np.float64)

dfgrd1 = np.array([
    [1.02000000000016      ,-1.452449271965897E-013 , 1.875825883512808E-013],
    [-1.620023559745261E-013,  0.945332212756261     ,  1.470136512526958E-012],
    [ 2.448527491871805E-013, -1.478511757468974E-012 ,  1.03699103875074]]
,dtype=np.float64)

F = dfgrd1.transpose()

In [133]:
a0 = np.array([1,0,0])
s0 = np.array([0,1,0])

In [141]:
J = np.linalg.det(F)
C = np.matmul(F.transpose(),F)
Chat = J**(-2/3)*C
detC = np.linalg.det(C)
Cinv = np.linalg.inv(C)
Fhat = J**(-1/3)*F
Chat2 = np.matmul(Chat,Chat)
trChat2 = Chat2[0,0]+Chat2[1,1]+Chat2[2,2]

I1hat = Chat[0,0]+Chat[1,1]+Chat[2,2]
I2hat = 1/2*(I1hat**2-trChat2)
Iahat = np.dot(a0, np.dot(Chat,a0))
Ishat = np.dot(s0, np.dot(Chat,s0))

in1 = (I1hat-meanI1)/stdI1
in2 = (I2hat-meanI2)/stdI2
in3 = (Iahat-meanIa)/stdIa
in4 = (Ishat-meanIs)/stdIs
inputs = np.zeros([1,4])
inputs[:,0] = in1
inputs[:,1] = in2
inputs[:,2] = in3
inputs[:,3] = in4

intensor = tf.Variable(inputs)
with tf.GradientTape(persistent=True) as t:
    NN_outputs = model(intensor)
    grads1 = NN_outputs[:,1]**2
    grads2 = NN_outputs[:,2]**2
    gradsa = NN_outputs[:,3]**2
    gradss = NN_outputs[:,4]**2

Psi1 = NN_outputs[0][1].numpy()**2
Psi2 = NN_outputs[0][2].numpy()**2
Psia = NN_outputs[0][3].numpy()**2
Psis = NN_outputs[0][4].numpy()**2

grad = t.jacobian(NN_outputs, intensor)[0,:,0,:]
Psi11 = grad[1,0].numpy()
Psi12 = grad[1,1].numpy()
Psi1a = grad[1,2].numpy()
Psi1s = grad[1,3].numpy()
Psi21 = grad[2,0].numpy()
Psi22 = grad[2,1].numpy()
Psi2a = grad[2,2].numpy()
Psi2s = grad[2,3].numpy()
Psia1 = grad[3,0].numpy()
Psia2 = grad[3,1].numpy()
Psiaa = grad[3,2].numpy()
Psias = grad[3,3].numpy()
Psis1 = grad[4,0].numpy()
Psis2 = grad[4,1].numpy()
Psisa = grad[4,2].numpy()
Psiss = grad[4,3].numpy()

#### Tensorflow stuff

print(gradPsi1, gradPsi2, gradPsia, gradPsis)

Psi1 = Psi1*stdPsi/stdI1
Psi2 = Psi2*stdPsi/stdI2
Psia = Psia*stdPsi/stdIa
Psis = Psis*stdPsi/stdIs

Psi11 = Psi11*stdPsi/stdI1/stdI1
Psi12 = Psi12*stdPsi/stdI1/stdI2
Psi1a = Psi1a*stdPsi/stdI1/stdIa
Psi1s = Psi1s*stdPsi/stdI1/stdIs
Psi21 = Psi21*stdPsi/stdI2/stdI1
Psi22 = Psi22*stdPsi/stdI2/stdI2
Psi2a = Psi2a*stdPsi/stdI2/stdIa
Psi2s = Psi2s*stdPsi/stdI2/stdIs
Psia1 = Psia1*stdPsi/stdIa/stdI1
Psia2 = Psia2*stdPsi/stdIa/stdI2
Psiaa = Psiaa*stdPsi/stdIa/stdIa
Psias = Psias*stdPsi/stdIa/stdIs
Psis1 = Psis1*stdPsi/stdIs/stdI1
Psis2 = Psis2*stdPsi/stdIs/stdI2
Psisa = Psisa*stdPsi/stdIs/stdIa
Psiss = Psiss*stdPsi/stdIs/stdIs

Psi12  = 0.5*(Psi12 + Psi21)
Psi1a  = 0.5*(Psi1a + Psia1)
Psi2a  = 0.5*(Psi2a + Psia2)
Psi1s  = 0.5*(Psi1s + Psis1)
Psi2s  = 0.5*(Psi2s + Psis2)
Psias  = 0.5*(Psias + Psisa)

tf.Tensor([[ 0.33348714  0.28588636 -0.00069868  0.25107797]], shape=(1, 4), dtype=float64) tf.Tensor([[ 0.25187445  0.21606486 -0.00064973  0.18949357]], shape=(1, 4), dtype=float64) tf.Tensor([[-4.44809972e-04 -4.43677445e-04  6.01339456e-05 -2.74186347e-04]], shape=(1, 4), dtype=float64) tf.Tensor([[ 0.13536345  0.11597636 -0.0002181   0.10197335]], shape=(1, 4), dtype=float64)


In [114]:
I = np.eye(3)
II = 0.5*(np.einsum('ik,jl->ijkl',I,I) + np.einsum('il,jk->ijkl',I,I))
A0 = np.outer(a0,a0)
S0 = np.outer(s0,s0)

In [115]:
Shat = 2*(Psi1*I + Psi2*(I1hat*I-Chat) + Psia*A0 + Psis*S0)

In [116]:
K = 10
Psivol = K*(J-1)**2
p = 2*K*(J-1)
dpdJ = 2*K
ptilde = p + J*dpdJ

In [117]:
PP1 = II-1.0/3.0*np.einsum('ij,kl->ijkl',Cinv,C)
Siso = np.einsum('ijkl,kl->ij',J**(-2.0/3.0)*PP1, Shat)
S = Siso + J*p*Cinv
sigma = 1/J*np.einsum('ij,jk,kl->il',F,S,F.transpose())

In [118]:
d1 = +2*(Psi11 + 2*I1hat*Psi12 + Psi2 + I1hat**2*Psi22)
d2 = -2*(Psi12 + I1hat*Psi22)
d3 = +2*(Psi1a + Psi2a)
d4 = +2*(Psi1s + Psi2s)
d5 = -2*Psi2a
d6 = -2*Psi2s
d7 = +2*Psias
d8 = +2*Psiaa
d9 = +2*Psiss

In [119]:
CChat = 2*J**(-4/3)*(
    d1*np.einsum('ij,kl->ijkl',I,I) + 
    d2*(np.einsum('ij,kl->ijkl',Chat,I   ) + np.einsum('ij,kl->ijkl',I   ,Chat)) +
    d3*(np.einsum('ij,kl->ijkl',A0  ,I   ) + np.einsum('ij,kl->ijkl',I   ,A0  )) +
    d4*(np.einsum('ij,kl->ijkl',S0  ,I   ) + np.einsum('ij,kl->ijkl',I   ,S0  )) +
    d5*(np.einsum('ij,kl->ijkl',A0  ,Chat) + np.einsum('ij,kl->ijkl',Chat,A0  )) +
    d6*(np.einsum('ij,kl->ijkl',S0  ,Chat) + np.einsum('ij,kl->ijkl',Chat,S0  )) +
    d7*(np.einsum('ij,kl->ijkl',A0  ,S0  ) + np.einsum('ij,kl->ijkl',S0  ,A0  )) +
    d8*np.einsum('ij,kl->ijkl',A0  ,A0  ) +
    d9*np.einsum('ij,kl->ijkl',S0  ,S0  )
)

In [120]:
CCvol = J*ptilde*np.einsum('ij,kl->ijkl',Cinv,Cinv) - 2*J*p*1/2*(np.einsum('ik,jl->ijkl',Cinv,Cinv) + np.einsum('il,jk->ijkl',Cinv,Cinv))

In [121]:
PP2 = 1/2*(np.einsum('ik,jl->ijkl',Cinv,Cinv) + np.einsum('il,jk->ijkl',Cinv,Cinv)) - 1/3*np.einsum('ij,kl->ijkl',Cinv,Cinv)
CCiso = -2/3*np.einsum('ij,kl->ijkl',Siso,Cinv) + J**(-4/3)*np.einsum('ijkl,klmn,opmn->ijop',PP1,CChat,PP1) - \
        2/3*np.einsum('ij,kl->ijkl',Cinv,Siso)  + 2/3*J**(-2/3)*(Shat[0,0]+Shat[1,1]+Shat[2,2])*PP2

In [122]:
CC = CCvol + CCiso

In [123]:
cc = 1/J*np.einsum('iI,jJ,kK,lL,IJKL->ijkl',F,F,F,F,CC)

In [124]:
Itoi = [0, 1, 2, 0, 0, 1]
Itoj = [0, 1, 2, 1, 2, 2]

ddsdde = np.zeros((6,6))
ID = np.eye(3)
for II in range(6):
    for JJ in range(6):
        i = Itoi[II]
        j = Itoj[II]
        k = Itoi[JJ]
        l = Itoj[JJ]
        ddsdde[II,JJ]+= cc[i,j,k,l] + 0.5*(ID[i,k]*sigma[j,l] + ID[i,l]*sigma[j,k] + ID[j,k]*sigma[i,l] + ID[j,l]*sigma[i,k])

In [125]:
#print(F,J,C,Chat,I1hat,I2hat,Iahat,Ishat,Shat,Siso,S,sigma,Psi2a,CChat,CCvol,CCiso,CC,ddsdde)
print('F')
print(F)
print('J')
print(J)
print('C')
print(C)
print('Cinv')
print(Cinv)
print('Chat')
print(Chat)
print('I1hat, I2hat, I4ahat, I4shat')
print(I1hat, I2hat, Iahat, Ishat)
print('Psi1', 'Psi2', 'Psia', 'Psis')
print(Psi1, Psi2, Psia, Psis)
print('Shat')
print(Shat)
print('PP1')
print(PP1)
print('Siso')
print(Siso)
print('S')
print(S)
print('sigma')
print(sigma)
print('CChat')
print(CChat)
print('ptilde')
print(ptilde)
print('p')
print(p)
print('CCvol')
print(CCvol)
print('CCiso')
print(CCiso)
print('CC_ref')
print(CC)
print('DDSDDE')
print(ddsdde)

F
[[ 1.02000000e+00 -1.62002356e-13  2.44852749e-13]
 [-1.45244927e-13  9.45332213e-01 -1.47851176e-12]
 [ 1.87582588e-13  1.47013651e-12  1.03699104e+00]]
J
0.9999070539362204
C
[[ 1.04040000e+00 -3.02547112e-13  4.44271267e-13]
 [-3.02547112e-13  8.93652992e-01  1.26833598e-13]
 [ 4.44271267e-13  1.26833598e-13  1.07535041e+00]]
Cinv
[[ 9.61168781e-01  3.25404649e-13 -3.97098162e-13]
 [ 3.25404649e-13  1.11900258e+00 -1.31982209e-13]
 [-3.97098162e-13 -1.31982209e-13  9.29929432e-01]]
Chat
[[ 1.04046447e+00 -3.02565860e-13  4.44298798e-13]
 [-3.02565860e-13  8.93708371e-01  1.26841458e-13]
 [ 4.44298798e-13  1.26841458e-13  1.07541705e+00]]
I1hat, I2hat, I4ahat, I4shat
3.0095898961695884 3.009914267528706 1.0404644723839014 0.89370837111605
Psi1 Psi2 Psia Psis
0.006529107795693995 0.004876827410086063 7.304586902637948e-06 0.0027025719236673275
Shat
[[ 3.22789944e-02  2.95112296e-15 -4.33353712e-15]
 [ 2.95112296e-15  3.91009375e-02 -1.23716780e-15]
 [-4.33353712e-15 -1.23716780e-15 